### 📥 Daten laden & erste Sichtprüfung

In dieser Zelle wird der Kreditkarten-Transaktionsdatensatz in einen Pandas DataFrame geladen.

**Ziele dieser Zelle:**
- Sicherstellen, dass die CSV-Datei korrekt gelesen wird
- Überprüfen der grundlegenden Struktur des Datensatzes
- Erste visuelle Kontrolle der Spalten und Werte

**Was passiert konkret:**
- `pandas` wird als zentrales Werkzeug für tabellarische Daten importiert
- Die CSV-Datei wird über einen **relativen Pfad** geladen (projektportabel)
- `df.shape` zeigt Anzahl der Zeilen (Transaktionen) und Spalten (Features)
- `df.head()` zeigt die ersten fünf Transaktionen zur visuellen Plausibilitätsprüfung

Diese Schritte sind ein obligatorischer Einstieg in jedes Data-Science-Projekt, bevor mit Analysen oder Modellen begonnen wird.


In [5]:
import pandas as pd

path = "../data/raw/creditcard.csv"
df = pd.read_csv(path)

print("Shape:", df.shape)
display(df.head())


Shape: (284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### ⚖️ Klassenverteilung analysieren (Fraud vs. Non-Fraud)

Diese Zelle analysiert die Verteilung der Zielvariable `Class`, also wie viele Transaktionen betrügerisch sind und wie viele nicht.

**Ziele dieser Zelle:**
- Quantifizieren der Klassen-Unbalance
- Verstehen, wie selten Fraud-Fälle tatsächlich sind
- Vorbereitung auf eine sinnvolle Modell-Evaluation

**Was passiert konkret:**
- `value_counts()` zählt die absolute Anzahl der Klassen (0 = legitim, 1 = Betrug)
- `value_counts(normalize=True)` berechnet die prozentuale Verteilung
- Die Ergebnisse werden explizit ausgegeben, um das Ausmaß der Unbalance sichtbar zu machen

**Wichtige Erkenntnis:**
Da Betrugsfälle extrem selten sind (< 1 %), ist Accuracy als Metrik ungeeignet.  
Dieses Ergebnis beeinflusst alle späteren Entscheidungen zu Modellwahl, Metriken und Thresholds.


In [2]:
class_counts = df["Class"].value_counts()
class_ratio = df["Class"].value_counts(normalize=True) * 100

print("Class counts:\n", class_counts)
print("\nClass ratio (%):\n", class_ratio.round(4))


Class counts:
 Class
0    284315
1       492
Name: count, dtype: int64

Class ratio (%):
 Class
0    99.8273
1     0.1727
Name: proportion, dtype: float64


### 💰⏱️ Statistische Zusammenfassung von Amount und Time

Diese Zelle untersucht die beiden **nicht-PCA-transformierten Features** `Amount` und `Time`.

**Ziele dieser Zelle:**
- Verstehen der Wertebereiche und Verteilungen
- Identifizieren von Skalierungsunterschieden zu den PCA-Features
- Vorbereitung auf Feature-Scaling im Preprocessing

**Was passiert konkret:**
- `describe()` liefert zentrale statistische Kennzahlen:
  - Minimum, Maximum
  - Mittelwert und Standardabweichung
  - Quartile (25 %, 50 %, 75 %)
- Die Ergebnisse werden separat für `Amount` und `Time` angezeigt

**Wichtige Erkenntnisse:**
- `Amount` ist stark rechtsschief verteilt (viele kleine, wenige große Beträge)
- `Time` ist ein technisches Zeitmerkmal (Sekunden seit erster Transaktion)
- Beide Features liegen auf völlig anderen Skalen als die PCA-Features  
→ **Feature Scaling ist zwingend erforderlich**


In [3]:
print("Amount summary:")
display(df["Amount"].describe())

print("\nTime summary:")
display(df["Time"].describe())


Amount summary:


count    284807.000000
mean         88.349619
std         250.120109
min           0.000000
25%           5.600000
50%          22.000000
75%          77.165000
max       25691.160000
Name: Amount, dtype: float64


Time summary:


count    284807.000000
mean      94813.859575
std       47488.145955
min           0.000000
25%       54201.500000
50%       84692.000000
75%      139320.500000
max      172792.000000
Name: Time, dtype: float64

### 🧪 Datenqualität & PCA-Feature-Sanity-Check

Diese Zelle überprüft die Datenqualität und bestätigt Eigenschaften der anonymisierten PCA-Features.

**Ziele dieser Zelle:**
- Prüfen, ob fehlende Werte vorhanden sind
- Bestätigen, dass die PCA-Features korrekt zentriert sind
- Sicherstellen, dass keine unerwarteten Ausreißer oder Datenprobleme vorliegen

**Was passiert konkret:**
- `isna().sum()` zählt fehlende Werte pro Spalte
- Die Spalten mit den meisten Missing Values werden angezeigt
- Alle PCA-Features (`V1` bis `V28`) werden gesammelt
- Für diese Features werden Minimum, Mittelwert und Maximum ausgegeben

**Wichtige Erkenntnisse:**
- Der Datensatz enthält **keine fehlenden Werte**
- PCA-Features sind erwartungsgemäß um 0 zentriert
- Skalen der PCA-Features sind vergleichbar, aber nicht identisch

Diese Prüfung bestätigt, dass der Datensatz technisch konsistent ist und ohne zusätzliche Bereinigung weiterverarbeitet werden kann.


In [6]:
print("Missing values per column:")
display(df.isna().sum().sort_values(ascending=False).head(10))

print("\nMin/Max of PCA-like features (V1-V28):")
v_cols = [c for c in df.columns if c.startswith("V")]
display(df[v_cols].describe().loc[["min", "mean", "max"]].T.head(10))


Missing values per column:


Time      0
V16       0
Amount    0
V28       0
V27       0
V26       0
V25       0
V24       0
V23       0
V22       0
dtype: int64


Min/Max of PCA-like features (V1-V28):


,min,mean,max
V1,-56.407510,1.168375e-15,2.454930
V2,-72.715728,3.416908e-16,22.057729
V3,-48.325589,-1.379537e-15,9.382558
V4,-5.683171,2.074095e-15,16.875344
V5,-113.743307,9.604066e-16,34.801666
V6,-26.160506,1.487313e-15,73.301626
V7,-43.557242,-5.556467e-16,120.589494
V8,-73.216718,1.213481e-16,20.007208
V9,-13.434066,-2.406331e-15,15.594995
V10,-24.588262,2.239053e-15,23.745136
